# AIリテラシ　第９回目
### AIによる予測
___
__買い物かごに関する分析__


データをColaboへドラッグ＆ドロップでアップロードする  
dataset_group.csv

### データの読み込み
データに列名が無い場合

In [ ]:
import pandas as pd
df = pd.read_csv('dataset_group.csv', header=None, names=['date', 'customer_id', 'item_name'])
df

### データの変換
商品毎のデータになっているものを、顧客毎に購入した商品をまとめる

In [ ]:
dataset = df.groupby('customer_id')['item_name'].apply(list)
dataset

### mlxtendをインストールする
機械学習やデータ分析等のタスクにおいて便利なツールが用意されたPythonライブラリ  
本家：http://rasbt.github.io/mlxtend/  
参考：https://qiita.com/altescy/items/60a6def66f13267f6347

In [ ]:
!pip install mlxtend

### 頻出商品の抽出
複数の顧客が購入することが多い商品を抽出してみる

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

te.columns_

In [ ]:
df2 = pd.DataFrame(te_ary, columns=te.columns_)
df2

In [ ]:
frequent_itemsets = apriori(df2, min_support=0.04, use_colnames=True)

frequent_itemsets

### アソシエーション分析の実行

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1)
rules = rules.sort_values('support', ascending = False).reset_index(drop=True)

display(rules)

| データ項目 | 項目概要 |
| ---- | ---- |
| antecedents | 前提条件（事象A） |
| consequents | 結果（事象B） |
| antecedent support | 全体の中で、事象Aが占める割合（＝事象Aの期待信頼度） |
| consequent support | 全体の中で、事象Bが占める割合（＝事象Bの期待信頼度） |
| support | 全体の中で、事象Aと事象Bの組み合わせが占める割合 |
| confidence | 事象Aを含むデータの中で、事象Bが占める確率 |
| lift | lift = confidence/consequent support |
| leverage | 事象Aと事象Bの観測頻度と、事象Aと事象Bが独立している場合に予想される頻度の差<br>leverage=0は完全な独立性を示す |
| conviction | convictionが高い場合、事象Bが事象Aとの相互性が高いことを指す |